<a href="https://colab.research.google.com/github/Tuan2001/person-re-ranking/blob/master/human_reid_dukemtmcreid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
%cd deep-person-reid/
!conda create --name torchreid python=3.7
!conda activate torchreid

# install dependencies
# make sure which python and which pip point to the correct path
!pip install -r requirements.txt

# install torch and torchvision (select the proper cuda version to suit your machine)
!conda install pytorch torchvision cudatoolkit=9.0 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 9809 (delta 188), reused 267 (delta 183), pack-reused 9526
Receiving objects: 100% (9809/9809), 9.59 MiB | 22.06 MiB/s, done.
Resolving deltas: 100% (7226/7226), done.
/content/deep-person-reid
/bin/bash: conda: command not found
/bin/bash: conda: command not found
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 185 kB 46.5 MB/s 
     |████████████████████████████████| 42 kB 768 kB/s 
     |████████████████████████████████| 42 kB 859 kB/s 
     |████████████████████████████████| 69 kB 6.7 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
/bin/b

In [ ]:
# before run this cell
# go to /content/deep-person-reid/torchreid/data/datasets/image/dukemtmcreid.py
# then change the dataset_url = 'http://188.138.127.15:81/Datasets/DukeMTMC-reID.zip'
import torchreid

In [ ]:
#prepare the data
datamanager = torchreid.data.ImageDataManager(
    root='$PATH_TO_DATA',
    sources='dukemtmcreid',
    targets='dukemtmcreid',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_erase'],
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
+ random erase
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/$PATH_TO_DATA/dukemtmc-reid"
* url="http://188.138.127.15:81/Datasets/DukeMTMC-reID.zip"
* destination="/content/deep-person-reid/$PATH_TO_DATA/dukemtmc-reid/DukeMTMC-reID.zip"
...100%, 154 MB, 3858 KB/s, 40 seconds passed
Extracting "/content/deep-person-reid/$PATH_TO_DATA/dukemtmc-reid/DukeMTMC-reID.zip"
DukeMTMCreID dataset is ready
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  galle

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------


  **************** Summary ****************
  source            : ['dukemtmcreid']
  # source datasets : 1
  # source ids      : 702
  # source images   : 16522
  # source cameras  : 8
  target            : ['dukemtmcreid']
  *****************************************




In [ ]:
#build model
model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
#build engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [ ]:
engine.run(
    save_dir='log/osnet_x1_0_dukemtmcreid_softmax_cosinelr',
    max_epoch=10,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/516]	time 0.479 (0.573)	data 0.001 (0.054)	eta 0:49:11	loss 6.4388 (6.5344)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/10][20/516]	time 0.482 (0.524)	data 0.000 (0.027)	eta 0:44:51	loss 6.6709 (6.5013)	acc 0.0000 (1.2500)	lr 0.000300
epoch: [1/10][30/516]	time 0.476 (0.507)	data 0.000 (0.019)	eta 0:43:22	loss 6.4941 (6.4842)	acc 3.1250 (1.4583)	lr 0.000300
epoch: [1/10][40/516]	time 0.478 (0.499)	data 0.000 (0.014)	eta 0:42:36	loss 6.3415 (6.4692)	acc 3.1250 (1.6406)	lr 0.000300
epoch: [1/10][50/516]	time 0.476 (0.495)	data 0.000 (0.011)	eta 0:42:08	loss 6.4272 (6.4624)	acc 6.2500 (1.8750)	lr 0.000300
epoch: [1/10][60/516]	time 0.478 (0.491)	data 0.000 (0.010)	eta 0:41:45	loss 6.1056 (6.4118)	acc 3.1250 (2.5000)	lr 0.000300
epoch: [1/10][70/516]	time 0.479 (0.489)	data 0.001 (0.008)	eta 0:41:29	loss 5.7443 (6.3787)	acc 3.1250 (2.8125)	lr 0.000300
epoch: [1/10][80/516]	time 0.475 (0.487)	data 0.000 (0.007)	eta 0:41:16	loss 5.8394 (6.3564)	acc 6.2500 (3.0469)	lr 0.000300


In [ ]:
#create visrank
engine.run(
    max_epoch = 10,
    eval_freq = 10,
    test_only = True,
    visrank = True,
    visrank_topk = 10 
)

##### Evaluating dukemtmcreid (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 2228-by-512 matrix
Extracting features from gallery set ...
Done, obtained 17661-by-512 matrix
Speed: 0.0755 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 62.7%
CMC curve
Rank-1  : 81.8%
Rank-5  : 90.4%
Rank-10 : 93.1%
Rank-20 : 95.1%
# query: 2228
# gallery 17661
Visualizing top-10 ranks ...
- done 100/2228
- done 200/2228
- done 300/2228
- done 400/2228
- done 500/2228
- done 600/2228
- done 700/2228
- done 800/2228
- done 900/2228
- done 1000/2228
- done 1100/2228
- done 1200/2228
- done 1300/2228
- done 1400/2228
- done 1500/2228
- done 1600/2228
- done 1700/2228
- done 1800/2228
- done 1900/2228
- done 2000/2228
- done 2100/2228
- done 2200/2228
Done. Images have been saved to "log/visrank_dukemtmcreid" ...


In [ ]:
!zip -r /content/deep-person-reid/visrank_dukemtmcreid /content/deep-person-reid/log/visrank_dukemtmcreid

In [ ]:
#create the actmap
!python tools/visualize_actmap.py \
--root $PATH_TO_DATA/dukemtmc-reid \
-d dukemtmcreid \
-m osnet_x1_0 \
--weights log/osnet_x1_0_dukemtmcreid_softmax_cosinelr/model \
--save-dir log/visactmap_osnet_x1_0_dukemtmcreid

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/dukemtmc-reid/dukemtmc-reid"
* url="http://188.138.127.15:81/Datasets/DukeMTMC-reID.zip"
* destination="/dukemtmc-reid/dukemtmc-reid/DukeMTMC-reID.zip"
...100%, 154 MB, 11864 KB/s, 13 seconds passed
Extracting "/dukemtmc-reid/dukemtmc-reid/DukeMTMC-reID.zip"
DukeMTMCreID dataset is ready
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
/usr/local/lib/python3.7/

In [ ]:
!zip -r /content/deep-person-reid/actmap_dukemtmcreid /content/deep-person-reid/log/visactmap_osnet_x1_0_dukemtmcreid/actmap_dukemtmcreid